# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!


- ref 1: DQN in Udacity: https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn
- ref 2: DQN by tnakae: https://github.com/tnakae/Udacity-DeepRL-p1-navigation/blob/master/TrainDQN.ipynb

## 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="/home/straight/Programming/JupyterNotebook/DRL nanodegree/P1 Navigation/Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print(brain_name)
print(env.brain_names)

BananaBrain
['BananaBrain']


## 2. Examine the State and Action Spaces
There is an agent that navigates in the enviornment with four actions at each time step:

- 0 - walk forward 
- 1 - walk backward 
- 2 - turn left 
- 3 - turn right

The state space has 37 dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction. A reward of +1 is provided for collecting a yellow banana, and a reward of -1 is provided for collecting a blue banana.

The task is episodic, and in order to solve the environment, the agent must get an average score of +13 over 100 consecutive episodes.
Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


## 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

### Choose the agent.

-hidden_layers (list of int)
Specify the structure of Neural Network (In: state, Out: estimated reward of each action)
If you set [64, 32]:
-Input : state
Hidden Layer 1 : 64 nodes
Hidden Layer 2 : 32 nodes
-Output : estimated reward of each action
-tau (float between 0 and 1)
update rate by soft update of target model
updated slower if you set smaller value.
learning_rate (float)
learning_rate used by Adam.

In [5]:
from dqn_agent import DQNAgent
agent = DQNAgent(state_size=state_size, action_size=action_size, seed=123)

### Initialization

In [12]:
# rest the enviornment
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0    

# initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break    

        
        
print("Score: {}".format(score))

Score: 1.0


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

## 4. Training.
Please use GPU to accelerate the training.

Run the code cell below to train the agent from scratch. You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

Alternatively, you can skip to the next step below (5. Watch a trained and smart Agent!), to load the saved model weights from a pre-trained agent.

- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
import numpy as np
import torch
from collections import deque

print("Cuda version:", torch.version.cuda)
print("pytorch version", torch.__version__)

Cuda version: 8.0.61
pytorch version 0.4.0


In [10]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Parameters: 
    ======
        n_episodes (int):    maximum number of training episodes
        max_t (int):         maximum number of timesteps per episode
        eps_start (float):   starting value of epsilon, for epsilon-greedy action selection
        eps_end (float):     minimum value of epsilon
        eps_decay (float):   multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon 
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]   # get the next state
        score = 0
        for t in range(max_t): # run for maximum of max_t
            action = agent.act(state, eps).astype(int)    # select the action
            
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        average_score = np.mean(scores_window)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if average_score >=13.0: # check if env is solved
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores



In [11]:
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]
scores = dqn()

RuntimeError: cublas runtime error : the GPU program failed to execute at /pytorch/aten/src/THC/THCBlas.cu:249

### plot the scores

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

## 5. Test the trained agent.
you will load the trained weights from file to watch a smart and trained agent!

In [13]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

In [14]:
for i in range(3):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]   # get the next state
    
    for j in range(200):
        action = agent.act(state).astype(int)          # selecct an action
        env_info = env.step(action)[brain_name]
        state = env_info.vector_observations[0]        # get the next state
        done = env_info.local_done[0]                  # see if episode has finished
        if done:
            break
            
print("Score: {}".format(score))

Score: 1.0


## 6. Close the program
When finished, you can close the environment.

In [ ]:
env.close()